In [ ]:
import numpy as np
import torch
import pickle
from ID3QNE_deepQnet import Dist_DQN

device = 'cuda' if torch.cuda.is_available() else 'cpu'
epochs = 1000

with open('requiredFile.pkl', 'rb') as f:
    D = pickle.load(f)

X = D['X_train'].to_numpy()
Xnext = D['Xnext_train'].to_numpy()
Action = D['Action_train'].to_numpy()
NextAction = D['NextAction_train']
Done = D['Done_train']
Bloc = D['Bloc_train']
SOFAS = D['SOFAS_train']
y = D['y_train'].to_numpy()

print("nbins:", D.get("nbins"))
print("Action stats:", np.min(Action), np.max(Action), "Unique:", np.unique(Action))
print("Reward stats:", np.unique(y, return_counts=True))

model = Dist_DQN(state_dim=X.shape[1], n_actions=len(np.unique(Action)))

batchs = {
    "state": X,
    "next_state": Xnext,
    "action": Action,
    "next_action": NextAction,
    "reward": y,
    "done": Done,
    "bloc_num": Bloc,
    "SOFAS": SOFAS,
}

open("training_log.txt", "w").close()
for epoch in range(epochs):
    # Prefer our method if present; otherwise, call teammate's .train(batchs, epoch)
    if hasattr(model, "train_model"):
        loss = model.train_model(batchs, epoch)
    elif hasattr(model, "train"):
        loss = model.train(batchs, epoch)  # use their signature
    else:
        raise RuntimeError("No valid training method found on model.")
    print(f"Epoch {epoch + 1}/{epochs} | Loss: {loss:.4f}")
    with open("training_log.txt", "a") as log:
        log.write(f"Epoch {epoch + 1} | Loss: {loss:.4f}\n")

print("Training complete.")
